In [4]:

import numpy as np
import keras
import librosa
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, LSTM, Reshape
from tensorflow.keras.optimizers import SGD
from sklearn import preprocessing
from keras.layers.core import Reshape
np.seterr(divide='ignore', invalid='ignore')


Using TensorFlow backend.
C:\Users\Dhruv Goyal\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Dhruv Goyal\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Dhruv Goyal\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Dhruv Goyal\anaconda3\lib\site-packages\tensorflow\py

{'divide': 'warn', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [5]:
def readAudio(filename):
    x, sr = librosa.load(filename, sr=16000)
    return x, sr

#calculate spectrogram
def calc_spec(x):
    n_fft = 1024
    hop_length = 512
    win_length = 1024
    X = np.abs(librosa.stft(x, n_fft = n_fft, hop_length = hop_length, win_length = win_length, window='hann', dtype = np.clongdouble))
    X = librosa.power_to_db(X**2,ref=np.max)
    return X

def saveSpectrogram(X, outfilename):
    assert outfilename[-4:]=='.npy'  #'outfilename extension should be .npy'
    np.save(outfilename, X)
    return

def readSpectrogram(infilename):
    X = np.load(infilename)
    return X

nmfcc = 13

In [6]:
def normalize(arr, features = 513):             # used to scale all values between 0 and 1
    arr_max = np.ndarray.max(arr, axis = 1)      #obtaining max and min values for each feature
    arr_min = np.ndarray.min(arr, axis = 1)
    arr_max = arr_max.reshape((features, 1))
    arr_min = arr_min.reshape((features, 1))
    diff = arr_max - arr_min                    #getting the range of each value
    arr = arr- arr_min
    arr = arr/diff 
    return arr

In [7]:
def createDataset(dirname):            # obtain 3d array (samples, 27, 19)                        
    speech, sr = readAudio(dirname)
    Speech = calc_spec(speech)          #obtain Spectrogram
    Speech = normalize(Speech)
    imageNo = Speech.shape[1]         #Number of samples (frames)
    X = np.zeros((imageNo, 27, 19))    
    for i in range(imageNo):
        Image = Speech[:, i]                # storing 513 features into Image 
        Image = Image.reshape((27, 19))       # reshaping 513 = 27*19 features
        X[i] = Image     
    return X

In [8]:
dirname = 'C:/Users/Dhruv Goyal/Desktop/Ye Khol/music2.wav'
X_music = createDataset(dirname)     #Music Dataset
imageNo = X_music.shape[0]
t1 =  np.array([[1,0,0]]*imageNo)    #Corresponding ground truths

In [9]:
dirname = 'C:/Users/Dhruv Goyal/Desktop/Ye Khol/Speech2.wav'
X_speech = createDataset(dirname)      #Speech Dataset
imageNo = X_speech.shape[0]
t2 =  np.array([[0,1,0]]*imageNo)  #Corresponding Ground Truths
print(t2.shape)
print(X_speech.shape[0])

(54392, 3)
54392


In [10]:
dirname = 'C:/Users/Dhruv Goyal/Desktop/Ye Khol/noise2.wav'
X_noise = createDataset(dirname)       #Silence Dataset
imageNo = X_noise.shape[0]
t3 =  np.array([[0,0,1]]*imageNo)      #Corresponding labels
print(t3.shape)
print(X_noise.shape[0])

(62501, 3)
62501


In [11]:
X_train = np.concatenate((X_music, X_speech, X_noise), axis = 0)       #concatenating music, speech and silence samples
X_train = np.expand_dims(X_train, axis=3)                         #adding a 4th dim since CNN models accept 4 dimensions
t_train = np.concatenate((t1, t2, t3), axis = 0)
print(X_train.shape)
print(t_train.shape)

(173104, 27, 19, 1)
(173104, 3)


In [17]:
model = Sequential()
model.add(Conv2D(64, (3, 3), activation = 'relu', input_shape = X_train.shape[1:]))   #convolution layers with kernel size of (3, 3)
model.add(Dropout(0.2))      #prevent overfitting

model.add(Conv2D(64, (3, 3), activation = 'relu'))    
model.add(Dropout(0.2))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(Dropout(0.2))

model.add(Flatten())    # convert data to vector which can be passed to dense layer

model.add(Dense(64, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(3, activation = 'softmax'))    #softmax for multi-class classification

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [18]:
opt = tf.keras.optimizers.Adam(learning_rate = 1e-3, decay = 1e-5)    #Adam is faster than stochastic gradient descent

In [19]:
model.compile(loss = 'categorical_crossentropy', optimizer = opt, metrics = ['accuracy'])    #loss used for multi-class classification

In [21]:
model.fit(X_train, t_train, epochs = 1, batch_size = 128)     #training model

173104/173104 [==============================] - 438s 3ms/sample - loss: 0.0528 - acc: 0.9835


In [22]:
# pathName = 'D:/Videh_Acads/IITK/5th Sem/EE603/Project/test_samples/test_sample-0.npy'

def print_cluster(pathName):
    Spect = readSpectrogram(pathName)     #function similar to createDataset
    Speech = normalize(Spect)
    imageNo = Speech.shape[1]
    X = np.zeros((imageNo, 27, 19))
    for i in range(imageNo):
        Image = Speech[:, i]
        Image = Image.reshape((27, 19))
        X[i] = Image

    X_test = X

    m_score = 0
    speech_score = 0

    classes = ['Music', 'Speech', 'Silence']
    cluster = []

    for i in range(X_test.shape[0]):
        X_temp = X_test[i]
#         print(X_temp.shape)
        X_temp = np.expand_dims(X_temp, axis=0)    # Since CNN accepts 4 dimensions
        X_temp = np.expand_dims(X_temp, axis=3) 
        arr = model.predict(X_temp)          #Predictions array
        cluster.append(classes[np.argmax(arr)])     # Select that class whose probability is most and append it to cluster

    return cluster   

In [28]:
def audioEventDetect(cluster, window_size = 0.064):     #initial event detection method
    super_list = []      #stores all events
    cur_list = []        #stores start time, end time and event class
    cur_list.append(0)
    cur_list.append(window_size)     # end time of first event
    cur_class = cluster[0]
    cur_list.append(cur_class)
    super_list.append(cur_list)
    prev_class = cur_class      
    cluster = cluster[1:]        #start iterating from 2nd element of cluster
    
    for cur_class in cluster:
        if cur_class == prev_class:     #eg. if music is followed by music 
            super_list[-1][1] += window_size - 0.032    #end time of current event is increased by window_size - 0.032s
        else:
            cur_list = super_list[-1]    
            new_start = cur_list[1]      #start time of new event is end time of previous event
            cur_list = [new_start]
            cur_list.append((new_start + window_size - 0.032))     # end time of new event is new start event + remaining part of overlapping window
            cur_list.append(cur_class)
            super_list.append(cur_list)
        prev_class = cur_class
                            
    return super_list

In [43]:
import glob
import csv

header1 = ['filename','event','onset','offset']
header2 = ['filename','Music','Speech']
countt = 0

file_paths = glob.glob('C:/Users/Dhruv Goyal/Desktop/mocktest_set/spectrogram' + '/**/*.npy',recursive = True)
for pathname in file_paths:
    filename = 'test_sample-' + str(countt)
    cluster = print_cluster(pathname)
#     print(cluster)
    events = audioEventDetect(cluster)
    
    noOfFrames = 29        # this functions does a majority voting on 28 (sub)frames and allots corresponding label to this new frame (0.96 s in length) of 28 (sub)frames 
    new_cluster = []
    count = 0 
    music_score = 0
    speech_score = 0
    silence_score = 0
    for title in cluster:     # count number of music, speech, or silence classes in 28 frames  
        if title == 'Music':
            music_score+= 1    
        elif title == 'Speech':
            speech_score+= 1 
        else:
            silence_score += 1
        if count == noOfFrames-1:     # iterate till 28 frames have been counted 
            count = -1
            if music_score > speech_score:
                if music_score > silence_score:
                    new_cluster.append('Music')    # max class is Music
                else:
                    new_cluster.append('Silence')   #max class is Silence.. and so on
            else:
                if speech_score > silence_score:
                    new_cluster.append('Speech')
                else:
                    new_cluster.append('Silence')
            music_score = 0
            speech_score = 0
            silence_score = 0 
        count+= 1
        
    events = audioEventDetect(new_cluster, 0.96)       #pass this new cluster of 0.96s frames through audio event detection method
    for event in events:
        if event[-1] == 'Silence':
            events.remove(event)
    print(filename)
    print(events)
    for event in events:
        data = [filename]
        data.append(event[2])
        data.append(event[0])
        data.append(event[1])
        with open('C:/Users/Dhruv Goyal/Desktop/event identification.csv', 'a',newline='') as f:
            writer = csv.writer(f)
            writer.writerow(data)  
    
#     print(events)
    m = 0
    s = 0
    for event in cluster:
        if event == 'Music':
            m=m+1
        elif event == 'Speech':
            s=s+1
    if m > 100:
        m = 1
    else:
        m =0
    if s > 100:
        s = 1
    else:
        s =0
    if m == 0:
        s=1
    
    data = [filename]
    data.append(m)
    data.append(s)
    with open('C:/Users/Dhruv Goyal/Desktop/event tagging.csv', 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(data)
    countt = countt + 1
    
    
    
    

test_sample-0
[[6.528, 9.312000000000001, 'Music']]
test_sample-1
[]
test_sample-2
[[2.816, 3.7439999999999998, 'Music'], [4.672, 5.6, 'Music']]
test_sample-3
[[0, 2.816, 'Music'], [5.6, 7.4559999999999995, 'Music'], [8.384, 9.312000000000001, 'Music']]
test_sample-4
[[0, 9.312000000000001, 'Music']]
test_sample-5
[]
test_sample-6
[[0, 9.312000000000001, 'Music']]
test_sample-7
[]
test_sample-8
[[0, 9.312000000000001, 'Music']]
test_sample-9
[[1.888, 3.7439999999999998, 'Speech'], [6.528, 8.384, 'Speech'], [8.384, 9.312000000000001, 'Music']]


In [ ]:
from keras.models import load_model
path = 'D:/Videh_Acads/IITK/5th Sem/EE603/Project/CNN_Model/my_model_CNN.h5'
path1 = 'C:/Users/Videh Aggarwal/Downloads/my_model_RNN.h5'
model.save(path)                 # save model
#model = load_model(path)        # load model